# 0. Import Dependencies

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import tensorflow as tf
import math

from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, classification_report
from tensorflow.keras.utils import to_categorical

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from tensorflow.keras.models import Sequential, Model

from tensorflow.keras.layers import (LSTM, Dense, Concatenate, Attention, Dropout, Softmax,
                                     Input, Flatten, Activation, Bidirectional, Permute, multiply, 
                                     ConvLSTM2D, MaxPooling3D, TimeDistributed, Conv2D, MaxPooling2D)

from scipy import stats

# disable some of the tf/keras training warnings 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "3"
tf.get_logger().setLevel("ERROR")
tf.autograph.set_verbosity(1)

# suppress untraced functions warning
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

2024-04-21 14:54:27.874532: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-21 14:54:27.920080: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-21 14:54:27.920124: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-21 14:54:27.921729: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-21 14:54:27.930514: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-21 14:54:27.931342: I tensorflow/core/platform/cpu_feature_guard.cc:1

# 5. Preprocess Data and Create Labels/Features

In [2]:
actions = np.array(['barbell biceps curl','bench press','chest fly machine','deadlift','decline bench press',
                  'hammer curl','hip thrust','incline bench press','lat pulldown','lateral raise',
                  'leg extension','leg raises','plank','pull Up','push-up','romanian deadlift',
                  'russian twist','shoulder press','squat','t bar row','tricep Pushdown','tricep dips'])

In [3]:
label_map = {label:num for num, label in enumerate(actions)}

In [4]:
import pickle

In [5]:
# Load and organize recorded training data
sequences, labels = [], []
result_dir="WVE"
for action in actions:
    output_dir = os.path.join(result_dir, action)
    result_file_path_x = output_dir + "+X.txt"

    videos_data = []
    with open(result_file_path_x, "rb") as filex:
      for inpx in pickle.load(filex):
        videos_data.append(inpx)
    
    for video_data in videos_data:
        sequences.append(video_data)
        labels.append(label_map[action])

In [6]:
s1 = sequences[0]
landmark_obj = s1[0]
for lo in landmark_obj:
    lo.x = 0
    lo.y = 0
    lo.z = 0
    lo.visibility = 0

In [7]:
fixed_seq_length = 250
num_landmarks = 33

sequences = [seq[:len(seq)] + [landmark_obj]  * (fixed_seq_length - len(seq)) if len(seq) < fixed_seq_length else seq[:fixed_seq_length] for seq in sequences]

In [8]:
def extract_keypoints(results):
    """
    Processes and organizes the keypoints detected from the pose estimation model 
    to be used as inputs for the exercise decoder models
    
    """
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results]).flatten()
    return pose

In [9]:
sequences = [[extract_keypoints(s) for s in seq] for seq in sequences]

In [10]:
sequence_length = 40

def split_sequence(sequence, label, new_sequences, new_labels):
    for i in range(0, len(sequence), sequence_length):
    # Check if the remaining elements are less than the partition_length
        if len(sequence[i:]) >= sequence_length:
            new_sequences.append(sequence[i:i + sequence_length])
            new_labels.append(label)

In [11]:
print(len(sequences))
print(len(labels))

652
652


In [12]:
new_sequences, new_labels = [], []
for sequence, label in zip(sequences, labels):
    split_sequence(sequence, label, new_sequences, new_labels)

In [13]:
print(len(new_sequences))
print(len(new_labels))

3912
3912


In [14]:
sequences = new_sequences
labels = new_labels

In [15]:
# Make sure first dimensions of arrays match
X = np.array(sequences)
y = to_categorical(labels).astype(int)
print(X.shape, y.shape)

(3912, 40, 132) (3912, 22)


In [16]:
# Split into training, validation, and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1)
print(X_train.shape, y_train.shape)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=15/90, random_state=2)

(3520, 40, 132) (3520, 22)


# 6. Build and Train Neural Networks

In [17]:
DATA_PATH = os.path.join(os. getcwd(),'data') 
print(DATA_PATH)

/home/dell/Desktop/fyp/HP/data


In [18]:
# Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.01)

# some hyperparamters
batch_size = 32
max_epochs = 40

## 6a. LSTM

In [19]:
# Videos are going to be this many frames in length


num_input_values = num_landmarks * 4

In [20]:
lstm = Sequential()
lstm.add(LSTM(128, return_sequences=True, activation='relu', input_shape=(sequence_length, num_input_values)))
lstm.add(LSTM(256, return_sequences=True, activation='relu'))
lstm.add(LSTM(128, return_sequences=False, activation='relu'))
lstm.add(Dense(128, activation='relu'))
lstm.add(Dense(64, activation='relu'))
lstm.add(Dense(actions.shape[0], activation='softmax'))
print(lstm.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 40, 128)           133632    
                                                                 
 lstm_1 (LSTM)               (None, 40, 256)           394240    
                                                                 
 lstm_2 (LSTM)               (None, 128)               197120    
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 22)                1430      
                                                                 
Total params: 751190 (2.87 MB)
Trainable params: 751190 

In [21]:
lstm.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
lstm.fit(X_train, y_train, batch_size=batch_size, epochs=max_epochs, validation_data=(X_val, y_val))

Epoch 1/40
92/92 [==============================] - 13s 108ms/step - loss: 4261619.5000 - categorical_accuracy: 0.0641 - val_loss: 3.0111 - val_categorical_accuracy: 0.0903
Epoch 2/40
92/92 [==============================] - 10s 108ms/step - loss: 235.5587 - categorical_accuracy: 0.0794 - val_loss: 2.9450 - val_categorical_accuracy: 0.0954
Epoch 3/40
92/92 [==============================] - 10s 112ms/step - loss: 43.2028 - categorical_accuracy: 0.1043 - val_loss: 2.8946 - val_categorical_accuracy: 0.0954
Epoch 4/40
92/92 [==============================] - 10s 113ms/step - loss: 2.9071 - categorical_accuracy: 0.1108 - val_loss: 2.8902 - val_categorical_accuracy: 0.1175
Epoch 5/40
92/92 [==============================] - 11s 116ms/step - loss: 2437.1882 - categorical_accuracy: 0.1074 - val_loss: 2.9798 - val_categorical_accuracy: 0.0733
Epoch 6/40
92/92 [==============================] - 11s 116ms/step - loss: 348.1482 - categorical_accuracy: 0.0951 - val_loss: 2.9626 - val_categorical_a

<h1>Some Random Model<h1/>

In [22]:
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Bidirectional, Dropout, GlobalAveragePooling1D
CnnBLSTM = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(sequence_length, num_input_values)),
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(128)),
    Dropout(0.5),
    Dense(actions.shape[0], activation='softmax')
])

print("compiling the model")
# Compile the model
CnnBLSTM.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

print("Training the model")
# Train the model
CnnBLSTM.fit(X_train, y_train, epochs=max_epochs, batch_size=32, validation_data=(X_val, y_val))

compiling the model
Training the model
Epoch 1/40
92/92 [==============================] - 15s 105ms/step - loss: 2.5759 - categorical_accuracy: 0.2260 - val_loss: 2.1117 - val_categorical_accuracy: 0.3441
Epoch 2/40
92/92 [==============================] - 9s 96ms/step - loss: 2.1056 - categorical_accuracy: 0.3075 - val_loss: 1.8717 - val_categorical_accuracy: 0.4072
Epoch 3/40
92/92 [==============================] - 9s 93ms/step - loss: 2.0060 - categorical_accuracy: 0.3454 - val_loss: 1.8910 - val_categorical_accuracy: 0.3816
Epoch 4/40
92/92 [==============================] - 9s 96ms/step - loss: 1.8734 - categorical_accuracy: 0.3931 - val_loss: 1.8866 - val_categorical_accuracy: 0.3969
Epoch 5/40
92/92 [==============================] - 9s 95ms/step - loss: 1.7852 - categorical_accuracy: 0.4200 - val_loss: 1.6609 - val_categorical_accuracy: 0.4600
Epoch 6/40
92/92 [==============================] - 9s 97ms/step - loss: 1.7134 - categorical_accuracy: 0.4408 - val_loss: 1.6299 - va

## 6b. LSTM + Attention

In [23]:
def attention_block(inputs, time_steps):
    """
    Attention layer for deep neural network
    
    """
    # Attention weights
    a = Permute((2, 1))(inputs)
    a = Dense(time_steps, activation='softmax')(a)
    
    # Attention vector
    a_probs = Permute((2, 1), name='attention_vec')(a)
    
    # Luong's multiplicative score
    output_attention_mul = multiply([inputs, a_probs], name='attention_mul') 
    
    return output_attention_mul

In [24]:
HIDDEN_UNITS = 256

# Input
inputs = Input(shape=(sequence_length, num_input_values))

# Bi-LSTM
lstm_out = Bidirectional(LSTM(HIDDEN_UNITS, return_sequences=True))(inputs)

# Attention
attention_mul = attention_block(lstm_out, sequence_length)
attention_mul = Flatten()(attention_mul)

# Fully Connected Layer
x = Dense(2*HIDDEN_UNITS, activation='relu')(attention_mul)
x = Dropout(0.5)(x)

# Output
x = Dense(actions.shape[0], activation='softmax')(x)

# Bring it all together
AttnLSTM = Model(inputs=[inputs], outputs=x)
print(AttnLSTM.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 40, 132)]            0         []                            
                                                                                                  
 bidirectional_2 (Bidirecti  (None, 40, 512)              796672    ['input_1[0][0]']             
 onal)                                                                                            
                                                                                                  
 permute (Permute)           (None, 512, 40)              0         ['bidirectional_2[0][0]']     
                                                                                                  
 dense_4 (Dense)             (None, 512, 40)              1640      ['permute[0][0]']         

In [25]:
AttnLSTM.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
AttnLSTM.fit(X_train, y_train, batch_size=batch_size, epochs=max_epochs, validation_data=(X_val, y_val))

Epoch 1/40
92/92 [==============================] - 23s 218ms/step - loss: 2.5579 - categorical_accuracy: 0.2131 - val_loss: 2.0462 - val_categorical_accuracy: 0.3560
Epoch 2/40
92/92 [==============================] - 19s 203ms/step - loss: 2.0800 - categorical_accuracy: 0.3103 - val_loss: 1.8846 - val_categorical_accuracy: 0.3714
Epoch 3/40
92/92 [==============================] - 19s 209ms/step - loss: 1.9204 - categorical_accuracy: 0.3628 - val_loss: 1.7911 - val_categorical_accuracy: 0.3935
Epoch 4/40
92/92 [==============================] - 19s 207ms/step - loss: 1.8040 - categorical_accuracy: 0.4020 - val_loss: 1.7376 - val_categorical_accuracy: 0.4344
Epoch 5/40
92/92 [==============================] - 19s 204ms/step - loss: 1.7461 - categorical_accuracy: 0.4197 - val_loss: 1.6862 - val_categorical_accuracy: 0.4497
Epoch 6/40
92/92 [==============================] - 20s 215ms/step - loss: 1.6706 - categorical_accuracy: 0.4426 - val_loss: 1.6312 - val_categorical_accuracy: 0.473

In [25]:
# Model map
models = {
    'LSTM': lstm, 
    'LSTM_Attention_128HUs': AttnLSTM, 
    'CNN_BLSTM model': CnnBLSTM,
}

# 7a. Save Weights

In [26]:
for model_name, model in models.items():
    save_dir = os.path.join(os.getcwd(), f"{model_name}.h5")
    model.save(save_dir)

/home/dell/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# 7b. Load Weights

In [27]:
# Run model rebuild before doing this
for model_name, model in models.items():
    load_dir = os.path.join(os.getcwd(), f"{model_name}.h5")
    model.load_weights(load_dir)

# 8. Make Predictions

In [28]:
for model in models.values():
    res = model.predict(X_test, verbose=0)   

# 9. Evaluations using Confusion Matrix and Accuracy

In [29]:
eval_results = {}
eval_results['confusion matrix'] = None
eval_results['accuracy'] = None
eval_results['precision'] = None
eval_results['recall'] = None
eval_results['f1 score'] = None

confusion_matrices = {}
classification_accuracies = {}   
precisions = {}
recalls = {}
f1_scores = {} 

## 9a. Confusion Matrices

In [30]:
for model_name, model in models.items():
    yhat = model.predict(X_test, verbose=0)
    
    # Get list of classification predictions
    ytrue = np.argmax(y_test, axis=1).tolist()
    yhat = np.argmax(yhat, axis=1).tolist()
    
    # Confusion matrix
    confusion_matrices[model_name] = multilabel_confusion_matrix(ytrue, yhat)
    print(f"{model_name} confusion matrix: {os.linesep}{confusion_matrices[model_name]}")

# Collect results 
eval_results['confusion matrix'] = confusion_matrices

LSTM confusion matrix: 
[[[  0 357]
  [  0  35]]

 [[360   0]
  [ 32   0]]

 [[369   0]
  [ 23   0]]

 [[364   0]
  [ 28   0]]

 [[386   0]
  [  6   0]]

 [[380   0]
  [ 12   0]]

 [[381   0]
  [ 11   0]]

 [[373   0]
  [ 19   0]]

 [[360   0]
  [ 32   0]]

 [[371   0]
  [ 21   0]]

 [[383   0]
  [  9   0]]

 [[377   0]
  [ 15   0]]

 [[387   0]
  [  5   0]]

 [[378   0]
  [ 14   0]]

 [[357   0]
  [ 35   0]]

 [[387   0]
  [  5   0]]

 [[384   0]
  [  8   0]]

 [[382   0]
  [ 10   0]]

 [[372   0]
  [ 20   0]]

 [[383   0]
  [  9   0]]

 [[358   0]
  [ 34   0]]

 [[383   0]
  [  9   0]]]
LSTM_Attention_128HUs confusion matrix: 
[[[243 114]
  [ 17  18]]

 [[360   0]
  [ 32   0]]

 [[186 183]
  [ 13  10]]

 [[364   0]
  [ 28   0]]

 [[386   0]
  [  6   0]]

 [[380   0]
  [ 12   0]]

 [[381   0]
  [ 11   0]]

 [[373   0]
  [ 19   0]]

 [[360   0]
  [ 32   0]]

 [[371   0]
  [ 21   0]]

 [[383   0]
  [  9   0]]

 [[356  21]
  [ 14   1]]

 [[380   7]
  [  5   0]]

 [[378   0]
  [ 14   0]]


## 9b. Accuracy

In [31]:
for model_name, model in models.items():
    yhat = model.predict(X_test, verbose=0)
    
    # Get list of classification predictions
    ytrue = np.argmax(y_test, axis=1).tolist()
    yhat = np.argmax(yhat, axis=1).tolist()
    
    # Model accuracy
    classification_accuracies[model_name] = accuracy_score(ytrue, yhat)    
    print(f"{model_name} classification accuracy = {round(classification_accuracies[model_name]*100,3)}%")

# Collect results 
eval_results['accuracy'] = classification_accuracies

LSTM classification accuracy = 8.929%
LSTM_Attention_128HUs classification accuracy = 8.163%
CNN_BLSTM model classification accuracy = 53.827%


## 9c. Precision, Recall, and F1 Score

In [32]:
for model_name, model in models.items():
    yhat = model.predict(X_test, verbose=0)
    
    # Get list of classification predictions
    ytrue = np.argmax(y_test, axis=1).tolist()
    yhat = np.argmax(yhat, axis=1).tolist()
    
    # Precision, recall, and f1 score
    report = classification_report(ytrue, yhat, target_names=actions, output_dict=True)
    
    precisions[model_name] = report['weighted avg']['precision']
    recalls[model_name] = report['weighted avg']['recall']
    f1_scores[model_name] = report['weighted avg']['f1-score'] 
   
    print(f"{model_name} weighted average precision = {round(precisions[model_name],3)}")
    print(f"{model_name} weighted average recall = {round(recalls[model_name],3)}")
    print(f"{model_name} weighted average f1-score = {round(f1_scores[model_name],3)}\n")

# Collect results 
eval_results['precision'] = precisions
eval_results['recall'] = recalls
eval_results['f1 score'] = f1_scores

/home/dell/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dell/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dell/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


LSTM weighted average precision = 0.008
LSTM weighted average recall = 0.089
LSTM weighted average f1-score = 0.015



/home/dell/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dell/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dell/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


LSTM_Attention_128HUs weighted average precision = 0.019
LSTM_Attention_128HUs weighted average recall = 0.082
LSTM_Attention_128HUs weighted average f1-score = 0.03

CNN_BLSTM model weighted average precision = 0.741
CNN_BLSTM model weighted average recall = 0.538
CNN_BLSTM model weighted average f1-score = 0.575



# 10. Choose Model to Test in Real Time

In [33]:
model = AttnLSTM
model_name = 'AttnLSTM'

# 11. Calculate Joint Angles & Count Reps

In [34]:
def calculate_angle(a,b,c):
    """
    Computes 3D joint angle inferred by 3 keypoints and their relative positions to one another
    
    """
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [35]:
def get_coordinates(landmarks, mp_pose, side, joint):
    """
    Retrieves x and y coordinates of a particular keypoint from the pose estimation model
         
     Args:
         landmarks: processed keypoints from the pose estimation model
         mp_pose: Mediapipe pose estimation model
         side: 'left' or 'right'. Denotes the side of the body of the landmark of interest.
         joint: 'shoulder', 'elbow', 'wrist', 'hip', 'knee', or 'ankle'. Denotes which body joint is associated with the landmark of interest.
    
    """
    coord = getattr(mp_pose.PoseLandmark,side.upper()+"_"+joint.upper())
    x_coord_val = landmarks[coord.value].x
    y_coord_val = landmarks[coord.value].y
    return [x_coord_val, y_coord_val]            

In [36]:
def viz_joint_angle(image, angle, joint):
    """
    Displays the joint angle value near the joint within the image frame
    
    """
    cv2.putText(image, str(int(angle)), 
                   tuple(np.multiply(joint, [640, 480]).astype(int)), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                        )
    return

In [37]:
def count_reps(image, current_action, landmarks, mp_pose):
    """
    Counts repetitions of each exercise. Global count and stage (i.e., state) variables are updated within this function.
    
    """

    global curl_counter, press_counter, squat_counter, curl_stage, press_stage, squat_stage
    
    if current_action == 'curl':
        # Get coords
        shoulder = get_coordinates(landmarks, mp_pose, 'left', 'shoulder')
        elbow = get_coordinates(landmarks, mp_pose, 'left', 'elbow')
        wrist = get_coordinates(landmarks, mp_pose, 'left', 'wrist')
        
        # calculate elbow angle
        angle = calculate_angle(shoulder, elbow, wrist)
        
        # curl counter logic
        if angle < 30:
            curl_stage = "up" 
        if angle > 140 and curl_stage =='up':
            curl_stage="down"  
            curl_counter +=1
        press_stage = None
        squat_stage = None
            
        # Viz joint angle
        viz_joint_angle(image, angle, elbow)
        
    elif current_action == 'press':
        
        # Get coords
        shoulder = get_coordinates(landmarks, mp_pose, 'left', 'shoulder')
        elbow = get_coordinates(landmarks, mp_pose, 'left', 'elbow')
        wrist = get_coordinates(landmarks, mp_pose, 'left', 'wrist')

        # Calculate elbow angle
        elbow_angle = calculate_angle(shoulder, elbow, wrist)
        
        # Compute distances between joints
        shoulder2elbow_dist = abs(math.dist(shoulder,elbow))
        shoulder2wrist_dist = abs(math.dist(shoulder,wrist))
        
        # Press counter logic
        if (elbow_angle > 130) and (shoulder2elbow_dist < shoulder2wrist_dist):
            press_stage = "up"
        if (elbow_angle < 50) and (shoulder2elbow_dist > shoulder2wrist_dist) and (press_stage =='up'):
            press_stage='down'
            press_counter += 1
        curl_stage = None
        squat_stage = None
            
        # Viz joint angle
        viz_joint_angle(image, elbow_angle, elbow)
        
    elif current_action == 'squat':
        # Get coords
        # left side
        left_shoulder = get_coordinates(landmarks, mp_pose, 'left', 'shoulder')
        left_hip = get_coordinates(landmarks, mp_pose, 'left', 'hip')
        left_knee = get_coordinates(landmarks, mp_pose, 'left', 'knee')
        left_ankle = get_coordinates(landmarks, mp_pose, 'left', 'ankle')
        # right side
        right_shoulder = get_coordinates(landmarks, mp_pose, 'right', 'shoulder')
        right_hip = get_coordinates(landmarks, mp_pose, 'right', 'hip')
        right_knee = get_coordinates(landmarks, mp_pose, 'right', 'knee')
        right_ankle = get_coordinates(landmarks, mp_pose, 'right', 'ankle')
        
        # Calculate knee angles
        left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
        right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
        
        # Calculate hip angles
        left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
        right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
        
        # Squat counter logic
        thr = 165
        if (left_knee_angle < thr) and (right_knee_angle < thr) and (left_hip_angle < thr) and (right_hip_angle < thr):
            squat_stage = "down"
        if (left_knee_angle > thr) and (right_knee_angle > thr) and (left_hip_angle > thr) and (right_hip_angle > thr) and (squat_stage =='down'):
            squat_stage='up'
            squat_counter += 1
        curl_stage = None
        press_stage = None
            
        # Viz joint angles
        viz_joint_angle(image, left_knee_angle, left_knee)
        viz_joint_angle(image, left_hip_angle, left_hip)
        
    else:
        pass

# 12. Test in Real Time

In [38]:
def prob_viz(res, actions, input_frame, colors):
    """
    This function displays the model prediction probability distribution over the set of exercise classes
    as a horizontal bar graph
    
    """
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):        
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [39]:
# 1. New detection variables
sequence = []
predictions = []
res = []
threshold = 0.5 # minimum confidence to classify as an action/exercise
current_action = ''

# Rep counter logic variables
curl_counter = 0
press_counter = 0
squat_counter = 0
curl_stage = None
press_stage = None
squat_stage = None

# Camera object
cap = cv2.VideoCapture(0)

# Video writer object that saves a video of the real time test
fourcc = cv2.VideoWriter_fourcc('M','J','P','G') # video compression format
HEIGHT = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # webcam video frame height
WIDTH = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # webcam video frame width
FPS = int(cap.get(cv2.CAP_PROP_FPS)) # webcam video fram rate 

video_name = os.path.join(os.getcwd(),f"{model_name}_real_time_test.avi")
out = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*"MJPG"), FPS, (WIDTH,HEIGHT))

# Set mediapipe model 
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detection
        image, results = mediapipe_detection(frame, pose)
        
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)        
        sequence.append(keypoints)      
        sequence = sequence[-sequence_length:]
              
        if len(sequence) == sequence_length:
            res = model.predict(np.expand_dims(sequence, axis=0), verbose=0)[0]           
            predictions.append(np.argmax(res))
            current_action = actions[np.argmax(res)]
            confidence = np.max(res)
            
        #3. Viz logic
            # Erase current action variable if no probability is above threshold
            if confidence < threshold:
                current_action = ''

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
            # Count reps
            try:
                landmarks = results.pose_landmarks.landmark
                count_reps(
                    image, current_action, landmarks, mp_pose)
            except:
                pass

            # Display graphical information
            cv2.rectangle(image, (0,0), (640, 40), colors[np.argmax(res)], -1)
            cv2.putText(image, 'curl ' + str(curl_counter), (3,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, 'press ' + str(press_counter), (240,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, 'squat ' + str(squat_counter), (490,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
         
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        
        # Write to video file
        if ret == True:
            out.write(image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()

NameError: name 'mp_pose' is not defined

In [40]:
cap.release()
out.release()
cv2.destroyAllWindows()